In [1]:
##############################################################################
#                                                                            #
#  Code for the USENIX Security '22 paper:                                   #
#  How Machine Learning Is Solving the Binary Function Similarity Problem.   #
#                                                                            #
#  MIT License                                                               #
#                                                                            #
#  Copyright (c) 2019-2022 Cisco Talos                                       #
#                                                                            #
#  Permission is hereby granted, free of charge, to any person obtaining     #
#  a copy of this software and associated documentation files (the           #
#  "Software"), to deal in the Software without restriction, including       #
#  without limitation the rights to use, copy, modify, merge, publish,       #
#  distribute, sublicense, and/or sell copies of the Software, and to        #
#  permit persons to whom the Software is furnished to do so, subject to     #
#  the following conditions:                                                 #
#                                                                            #
#  The above copyright notice and this permission notice shall be            #
#  included in all copies or substantial portions of the Software.           #
#                                                                            #
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,           #
#  EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF        #
#  MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                     #
#  NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE    #
#  LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION    #
#  OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION     #
#  WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.           #
#                                                                            #
#  Dataset-1 sanity check                                                    #
#                                                                            #
##############################################################################

In [2]:
## Requirements
# pandas==1.4.2

In [3]:
import json
import os
import pandas as pd
import unittest

In [4]:
N_SELECTED_FUNCS_TRAINING = 256625
N_SELECTED_BINARIES_TRAINING = 2978

N_SELECTED_FUNCS_VALIDATION = 12736
N_SELECTED_BINARIES_VALIDATION = 859

N_SELECTED_FUNCS_TESTING = 522003
N_SELECTED_BINARIES_TESTING = 919

In [5]:
!ls .

Dataset-1 plots.ipynb        testing_Dataset-1.csv
Dataset-1 sanity check.ipynb training_Dataset-1.csv
features                     validation_Dataset-1.csv
pairs


In [6]:
class TestDataset1(unittest.TestCase):
    
    def setUp(self):
        training_dataset = pd.read_csv("training_Dataset-1.csv", index_col=0)
        self.fva_set_training = set([tuple(x) for x in training_dataset[['idb_path', 'fva']].values])
        self.idb_set_training = set(training_dataset['idb_path'].values)
        
        validation_dataset = pd.read_csv("validation_Dataset-1.csv", index_col=0)
        self.fva_set_validation = set([tuple(x) for x in validation_dataset[['idb_path', 'fva']].values])
        self.idb_set_validation = set(validation_dataset['idb_path'].values)

        testing_dataset = pd.read_csv("testing_Dataset-1.csv", index_col=0)
        self.fva_set_testing = set([tuple(x) for x in testing_dataset[['idb_path', 'fva']].values])
        self.idb_set_testing = set(testing_dataset['idb_path'].values)
        
    def test_training_dataset(self):
        training_dataset = pd.read_csv("training_Dataset-1.csv", index_col=0)

        # Test null values
        self.assertFalse(training_dataset.isnull().values.any())

        # Test number of unique functions
        self.assertEqual(training_dataset.shape[0], N_SELECTED_FUNCS_TRAINING)
        fva_set_training = set([tuple(x) for x in training_dataset[['idb_path', 'fva']].values])
        self.assertEqual(len(fva_set_training), N_SELECTED_FUNCS_TRAINING)

        # Test the number of unique binaries
        idb_set_training = set(training_dataset['idb_path'].values)
        self.assertEqual(len(idb_set_training), N_SELECTED_BINARIES_TRAINING)
    
    def test_validation_dataset(self):
        validation_dataset = pd.read_csv("validation_Dataset-1.csv", index_col=0)
        
        # Test null values
        self.assertFalse(validation_dataset.isnull().values.any())
        
        # Test number of unique functions
        self.assertEqual(validation_dataset.shape[0], N_SELECTED_FUNCS_VALIDATION)
        fva_set_validation = set([tuple(x) for x in validation_dataset[['idb_path', 'fva']].values])
        self.assertEqual(len(fva_set_validation), N_SELECTED_FUNCS_VALIDATION)
        
        # Test the number of unique binaries
        idb_set_validation = set(validation_dataset['idb_path'].values)
        self.assertEqual(len(idb_set_validation), N_SELECTED_BINARIES_VALIDATION)
    
    def test_testing_dataset(self):
        testing_dataset = pd.read_csv("testing_Dataset-1.csv", index_col=0)

        # Test null values
        self.assertFalse(testing_dataset.isnull().values.any())

        # Test number of unique functions
        self.assertEqual(testing_dataset.shape[0], N_SELECTED_FUNCS_TESTING)
        fva_set_testing = set([tuple(x) for x in testing_dataset[['idb_path', 'fva']].values])
        self.assertEqual(len(fva_set_testing), N_SELECTED_FUNCS_TESTING)

        # Test the number of unique binaries
        idb_set_testing = set(testing_dataset['idb_path'].values)
        self.assertEqual(len(idb_set_testing), N_SELECTED_BINARIES_TESTING)
        
    def test_flowchart_dataset(self):
        flowchart_dataset = pd.read_csv("features/flowchart_Dataset-1.csv")

        # Test null values
        self.assertFalse(flowchart_dataset.isnull().values.any())

        # Functions in testing_dataset must a subset of those in flowchart
        fc_fva_set = set([tuple(x) for x in flowchart_dataset[['idb_path', 'fva']].values])
        self.assertEqual(self.fva_set_training & fc_fva_set, self.fva_set_training)
        self.assertEqual(self.fva_set_validation & fc_fva_set, self.fva_set_validation)
        self.assertEqual(self.fva_set_testing & fc_fva_set, self.fva_set_testing)
    
    def test_pairs_dataset_testing(self):
        for pair_name in os.listdir("pairs/testing/"):
            if not pair_name.endswith(".csv"):
                continue
            pair_path = os.path.join("pairs/testing/", pair_name)

            pairs_dataset = pd.read_csv(pair_path, index_col=0)

            # Test null values
            self.assertFalse(pairs_dataset.isnull().values.any())

            # Test pairs size            
            dt = dict(pairs_dataset.groupby("db_type").count()['idb_path_1'].items())
            if pair_name.startswith("pos_rank"):
                self.assertDictEqual(dt, {'XA': 200, 'XC': 200, 'XC+XB': 200, 'XM': 200})
            elif pair_name.startswith("neg_rank"):
                self.assertDictEqual(dt, {'XA': 20000, 'XC': 20000, 'XC+XB': 20000, 'XM': 20000})
            else:
                self.assertDictEqual(dt, {'XA': 50000, 'XC': 50000, 'XC+XB': 50000, 'XM': 50000, 'arch': 50000, 'bit': 50000, 'comp': 50000, 'opt': 50000, 'ver': 50000})

            # Test overlapping functions
            p_fva_set = set([tuple(x) for x in pairs_dataset[['idb_path_1', 'fva_1']].values])
            p_fva_set |= set([tuple(x) for x in pairs_dataset[['idb_path_2', 'fva_2']].values])
            self.assertEqual(p_fva_set & self.fva_set_testing, p_fva_set)
    
    def test_pairs_dataset_validation(self):
        for pair_name in os.listdir("pairs/validation/"):
            if not pair_name.endswith(".csv"):
                continue
            pair_path = os.path.join("pairs/validation/", pair_name)

            pairs_dataset = pd.read_csv(pair_path, index_col=0)

            # Test null values
            self.assertFalse(pairs_dataset.isnull().values.any())

            # Test pairs size            
            dt = dict(pairs_dataset.groupby("db_type").count()['idb_path_1'].items())
            self.assertDictEqual(dt, {'XA': 10000, 'XC': 10000, 'XC+XB': 10000, 'XM': 10000})

            # Test overlapping functions
            p_fva_set = set([tuple(x) for x in pairs_dataset[['idb_path_1', 'fva_1']].values])
            p_fva_set |= set([tuple(x) for x in pairs_dataset[['idb_path_2', 'fva_2']].values])
            self.assertEqual(p_fva_set & self.fva_set_validation, p_fva_set)
        
    def test_selected(self):
        with open("features/training/selected_training_Dataset-1.json") as f_in:
            selected = json.load(f_in)
            
            # Test overlapping functions
            entries_s = set([(k, hex(v)) for k in selected for v in selected[k]])
            self.assertEqual(entries_s & self.fva_set_training, entries_s | self.fva_set_training)
        
        with open("features/validation/selected_validation_Dataset-1.json") as f_in:
            selected = json.load(f_in)
            
            # Test overlapping functions
            entries_s = set([(k, hex(v)) for k in selected for v in selected[k]])
            self.assertEqual(entries_s & self.fva_set_validation, entries_s | self.fva_set_validation)
        
        with open("features/testing/selected_testing_Dataset-1.json") as f_in:
            selected = json.load(f_in)
            
            # Test overlapping functions
            entries_s = set([(k, hex(v)) for k in selected for v in selected[k]])
            self.assertEqual(entries_s & self.fva_set_testing, entries_s | self.fva_set_testing)

    def test_binary_files(self):
        a_list = [self.idb_set_training, self.idb_set_validation, self.idb_set_testing]
        b_list = [N_SELECTED_BINARIES_TRAINING, N_SELECTED_BINARIES_VALIDATION, N_SELECTED_BINARIES_TESTING]
        for bin_list, n_binaries in zip (a_list, b_list):
            binary_counter = 0
            for path in bin_list:
                npath = path.replace("IDBs/", "../../Binaries/")
                npath = npath.replace(".i64", "")
                if os.path.isfile(npath):
                    binary_counter += 1
            self.assertEqual(binary_counter, n_binaries)
    
    def test_acfg_disasm(self):
        a_list = [self.idb_set_training, self.idb_set_validation, self.idb_set_testing]
        b_list = [N_SELECTED_BINARIES_TRAINING, N_SELECTED_BINARIES_VALIDATION, N_SELECTED_BINARIES_TESTING]
        c_list = ['training', 'validation', 'testing']
        for bin_list, n_binaries, folder in zip (a_list, b_list, c_list):
            j_counter = 0
            for path in bin_list:
                path = os.path.join(
                    "features/{}/acfg_disasm_Dataset-1_{}".format(folder, folder),
                    os.path.basename(path).replace(".i64", "_acfg_disasm.json"))
                if os.path.isfile(path):
                    j_counter += 1
            self.assertEqual(j_counter, n_binaries)
    
    def test_acfg_features(self):
        a_list = [self.idb_set_training, self.idb_set_validation, self.idb_set_testing]
        b_list = [N_SELECTED_BINARIES_TRAINING, N_SELECTED_BINARIES_VALIDATION, N_SELECTED_BINARIES_TESTING]
        c_list = ['training', 'validation', 'testing']
        for bin_list, n_binaries, folder in zip (a_list, b_list, c_list):
            j_counter = 0
            for path in bin_list:
                path = os.path.join(
                    "features/{}/acfg_features_Dataset-1_{}".format(folder, folder),
                    os.path.basename(path).replace(".i64", "_acfg_features.json"))
                if os.path.isfile(path):
                    j_counter += 1
            self.assertEqual(j_counter, n_binaries)
    
    def test_fss(self):
        a_list = [self.idb_set_training, self.idb_set_validation, self.idb_set_testing]
        b_list = [N_SELECTED_BINARIES_TRAINING, N_SELECTED_BINARIES_VALIDATION, N_SELECTED_BINARIES_TESTING]
        c_list = ['training', 'validation', 'testing']
        for bin_list, n_binaries, folder in zip (a_list, b_list, c_list):
            j_counter = 0
            for path in bin_list:
                path = os.path.join(
                    "features/{}/fss_Dataset-1_{}".format(folder, folder),
                    os.path.basename(path).replace(".i64", "_Capstone_True_fss.json"))
                if os.path.isfile(path):
                    j_counter += 1
            self.assertEqual(j_counter, n_binaries)

unittest.main(argv=[''], verbosity=2, exit=False)

test_acfg_disasm (__main__.TestDataset1) ... ok
test_acfg_features (__main__.TestDataset1) ... ok
test_binary_files (__main__.TestDataset1) ... ok
test_flowchart_dataset (__main__.TestDataset1) ... ok
test_fss (__main__.TestDataset1) ... ok
test_pairs_dataset_testing (__main__.TestDataset1) ... ok
test_pairs_dataset_validation (__main__.TestDataset1) ... ok
test_selected (__main__.TestDataset1) ... ok
test_testing_dataset (__main__.TestDataset1) ... ok
test_training_dataset (__main__.TestDataset1) ... ok
test_validation_dataset (__main__.TestDataset1) ... ok

----------------------------------------------------------------------
Ran 11 tests in 70.876s

OK
